Automação Web e Busca de Informações com Python
Desafio:
Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:

Dólar
Euro
Ouro
Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

Usaremos o selenium
Importante: baixar o webdriver

In [65]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import sys

In [66]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
navegador = webdriver.Chrome('chromedriver',options=chrome_options)

In [67]:
# Passo 1: Pegar cotação dolár
navegador.get("https://www.google.com/")
navegador.find_element(
    By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar real')
navegador.find_element(
    By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro real")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

5.0173
5.515
324.63


In [68]:
# Passo 4: Importar a lista de produtos
tabela = pd.read_excel('/content/drive/MyDrive/intensivao_python/aula3/Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [69]:
# Passo 5: Recalcular o preço de cada produto
tabela.loc[tabela['Moeda']=="Dólar", 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda']=="Euro", 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda']=="Ouro", 'Cotação'] = float(cotacao_ouro)
display(tabela)

# Passo 6: Atualizar os preços
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0173,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.5150,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.0173,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.0173,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.5150,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.0173,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,324.6300,7000.00,1.15,8050.000


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0173,5017.249827,1.40,7024.149758
1,Carro Renault,4500.00,Euro,5.5150,24817.500000,2.00,49635.000000
2,Notebook Dell,899.99,Dólar,5.0173,4515.519827,1.70,7676.383706
3,IPhone,799.00,Dólar,5.0173,4008.822700,1.70,6814.998590
4,Carro Fiat,3000.00,Euro,5.5150,16545.000000,1.90,31435.500000
5,Celular Xiaomi,480.48,Dólar,5.0173,2410.712304,2.00,4821.424608
6,Joia 20g,20.00,Ouro,324.6300,6492.600000,1.15,7466.490000


In [70]:
# Passo 7: Salvar os novos preços dos produtos
tabela.to_excel('/content/drive/MyDrive/intensivao_python/aula3/produtos_novo.xlsm', index=False)

In [70]:
!pip install selenium
!pip install webdriver-manager
!apt-get update
!apt install chromium-chromedriver